In [1]:
import json
from os import listdir
from os.path import isfile, join
import altair as alt

In [2]:
import pandas as pd
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')

#### Load in Data
* Open all json files that contain traces and append them to a list with all of the traces.
    - This makes the data ready to use with pandas dataframes and easy to create visualizations with Altair


In [4]:
baseDirectory = 'data/synthetic/20210302-hipster-shop'
directories = listdir(baseDirectory)

FileNotFoundError: [Errno 2] No such file or directory: 'data/synthetic/20210302-hipster-shop'

In [ ]:
traces = []
for directory in directories:
    thisDirectory = baseDirectory + '/' + directory
    try:
        with open(thisDirectory) as f:
          data = json.load(f)
    except:
        continue
    traces.append(data)

In [ ]:
names = []
durations = []
traceIDs = []
parentIDs = []
ids = []
traceErrors = []
for trace in traces:
    
    for element in trace:
        traceIDs.append(element['traceId'])
        ids.append(element['id'])
        try:
            parentIDs.append(element['parentId'])
        except:
            parentIDs.append('Na')
        try:
            traceErrors.append(element['tags']['error'])
        except:
            traceErrors.append('false')
            
        names.append(element['name'])
        durations.append(element['duration'])

In [ ]:
traceDf = pd.DataFrame({'Resource Name' : names, 'Duration':durations, 'Trace_ID': traceIDs, 'ID': ids, 'Parent_ID': parentIDs, 'Error?':traceErrors})
traceDf

In [ ]:
traceSum = 0 
currTraceID = ''
duration_starts = []
duration_ends = []
duration_start = 0
duration_end = 0
for i in range(len(traceDf)):
    val = traceDf.iloc[i]
    if val['Trace_ID'] != currTraceID:
        duration_start = 0
        currTraceID = val['Trace_ID']
    else:
        duration_start = duration_end
    duration_end = duration_start +  val['Duration']
    duration_starts.append(duration_start)
    duration_ends.append(duration_end)

In [ ]:
traceDf['duration_start'] = duration_starts
traceDf['duration_end'] = duration_ends
traceDf['Data Transfered'] = [round(np.random.uniform(4,10000),1) for i in range(len(traceDf))]
traceDf.head(2)

In [ ]:
updatedTraces = pd.DataFrame()
for traceID in set(traceDf['Trace_ID']):
#     print(traceID)
    tes = traceDf[traceDf['Trace_ID'] == traceID]
    root = list(tes['Trace_ID'])[0]
    tes['order'] = [100 for x in range(len(tes))]
    tes['loc start'] = [100 for x in range(len(tes))]
    tes['loc end'] = [100 for x in range(len(tes))]

    tes
    order = []
    root = list(tes['Trace_ID'])[0]
    startLoc = 0
    order = 0
    for i in range(len(tes)):
        t = tes.iloc[i]
        if t['ID'] == root:
            t['order'] = order
            t['loc start'] = startLoc
            t['loc end'] = t['loc start'] + t['Duration']
            tes.iloc[i] = t
    c = 0
    currentRoot = root
    startLoc += 100000
    lastRoot = root
    lastRoots = []
    while c < len(tes) - 1:
        startLoc = tes[tes['ID'] == currentRoot]['loc start'] + 100000
        order = tes[tes['ID'] == currentRoot]['order']
        for i in range(len(tes)):
            t = tes.iloc[i]

            if t['Parent_ID'] == currentRoot:

                t['order'] = order + .5
                order = order + 1
                t['loc start'] = startLoc
                t['loc end'] = t['loc start'] + t['Duration']
                tes.iloc[i] = t
                lastRoots.append(t['ID'])
                c += 1
    #     order = order + 1
        currentRoot = lastRoots.pop(0)
#     print('done')
    updatedTraces = updatedTraces.append(tes)

SINGLE TRACE PREPROCESSING DONE, UPLOAD CSV TO "20210302-hipster-shop-singletrace.csv"

In [ ]:
traceDf = updatedTraces
traceDf.to_csv('20210302-hipster-shop-singletrace.csv')

In [ ]:
# def plotTrace(traceID):
#     source = traceDf[traceDf['Trace_ID'] == traceID]
#     bars = alt.Chart(source, title='Trace: ' + traceID).mark_bar().encode(
#         y=alt.Y('Resource Name', type='nominal', sort=None),
#         x = alt.X("duration_start:Q", title= "Duration"),
#         x2 = "duration_end:Q",
#         color = 'Resource Name:N',
#         tooltip = ['Duration:Q', 'Data Transfered:Q','Error?:N']
#     )
#     text = bars.mark_text(
#         align='left',
#         baseline='middle',
#         color = 'black',
#         dx=-3  # Nudges text to right so it doesn't appear on top of the bar
#     ).encode(
#         color = 'Error?:N'
#         text='Error?:N'
#     )
#     return (bars + text).properties(height=300)

In [ ]:
traceDf['loc start']//100000

In [ ]:
traceDf['depth'] =traceDf['loc start']//100000
traceDf.to_csv('20210302-hipster-shop-singletrace.csv')

In [ ]:
def plotTrace(traceID):
    source = traceDf[traceDf['Trace_ID'] == traceID]
    return alt.Chart(source, title='Waterfall View Trace: ' + traceID).mark_bar().encode(
        y=alt.Y('Resource Name', type='ordinal', sort=None),
        x = alt.X("duration_start:Q", title= "Duration"),
        x2 = "duration_end:Q",
        color = alt.Color('Error?:N',legend =None),
        tooltip = ['Duration:Q', 'Data Transfered:Q','Error?:N']
    )

In [ ]:
def plotTraceTree(traceID):
    source = traceDf[traceDf['Trace_ID'] == traceID]
    bars = alt.Chart(source, title='Tree View Trace: ' + traceID).mark_bar().encode(
        y=alt.Y('order:N', title='Resource Name', axis = None),
        x = alt.X("loc start:Q", title= "Duration"),
        x2= "loc end:Q",
        color = alt.Color('depth:Q',sort='descending', legend = alt.Legend(title = 'Tree Depth')),
        tooltip = ['ID:N', 'Parent_ID:N', 'Duration:Q','Error?:N']
    )
    
    text = alt.Chart(source).mark_text(
        align='right'
    ).encode(
        y=alt.Y('order:N', axis = None),
        x=alt.value(-3),
        text='Resource Name:N',
    )
    return bars + text

In [ ]:
plotTraceTree('00915d5693a26f30')

In [ ]:
plotTrace('ffea94949d425fe9')

In [ ]:
traceDf[traceDf['Trace_ID'] =='00915d5693a26f30']

In [ ]:
plotTraceTree('00915d5693a26f30')

In [ ]:
plotTrace('b9ae10ad77e3ecee')

In [ ]:
plotTraceTree('b9ae10ad77e3ecee')

In [ ]:
plotTrace('106b33d6227a4e20')

In [ ]:
plotTraceTree('106b33d6227a4e20')

AGGREGATE

In [ ]:
rootTraces = traceDf[traceDf['Parent_ID'] == 'Na']
rootTraces 

In [ ]:
roots = list(set(rootTraces['Resource Name']))
roots 

In [ ]:
avgDurations = []
errorPercentage = []
aggregateTraceDf = pd.DataFrame()
for root in roots:
    print(root)
    traces = rootTraces[rootTraces['Resource Name'] == root]
    ids = list(set(traces['Trace_ID']))
    print(len(traces))
    traceDf['keep'] = [True if x in ids else False for x in traceDf['Trace_ID']]
    thisRootTraces = traceDf[traceDf['keep']]
    print(len(thisRootTraces))
    thisRootTraces['intErrors?'] = [1 if x =='true' else 0 for x in thisRootTraces['Error?']]
    grouped = thisRootTraces.groupby('Resource Name').agg({'Duration': ['mean'], 'intErrors?':['sum','count'], 'Data Transfered':['mean'], 'loc start': 'mean','loc end': 'mean', 'order':'mean' })
    grouped = grouped.reset_index()
    grouped['root'] = [root] * len(grouped)
    cols = []
    for x in grouped.columns:
        if x[1] =='mean':
            grouped[x] = round(grouped[x],1)
            cols.append('Average ' + x[0])
        elif x[1] != '':
            cols.append(x[1])
        else:
            cols.append(x[0])
    grouped.columns = cols
    grouped['Error Rate'] = round(grouped['sum']/grouped['count'],3)
    grouped = grouped.rename(columns = {'mean': 'Average Duration', 'count':'Error Count'})
    cols = ['root','Resource Name', 'Average Duration','Error Count','Error Rate','Average Data Transfered', 'Average loc start', 'Average loc end',
           'Average order']
    grouped = grouped[cols]
    grouped    
    aggregateTraceDf = aggregateTraceDf.append(grouped)



AGGREGATE TRACE PREPROCESSING DONE, UPLOAD CSV TO "20210302-hipster-shop-aggregate.csv"

In [ ]:
aggregateTraceDf.to_csv('20210302-hipster-shop-aggregate.csv')

In [11]:
aggregateTraceDf = pd.read_csv('data/synthetic/20210409-hipster-shop-sl-agg.csv')
aggregateTraceDf 

,Unnamed: 0,root,Resource Name,Average Duration,Error Count,Error Rate,Average Data Transfered,Average loc start,Average loc end,Average order
0,0,/product,/adrequest,245798.4,1513,0.000,5038.5,100000,345798.4,1.5
1,1,/product,/getproducts,116964.6,3026,0.000,5025.3,150000,266964.6,1.8
2,2,/product,/getrecommendations,316405.2,1513,0.000,5049.7,100000,416405.2,1.5
3,3,/product,/product,569647.1,1513,0.000,4954.7,0,569647.1,0.0
4,0,/shipping,/address,51654.8,252,0.000,5149.0,200000,251654.8,1.0
5,1,/shipping,/getquote,298452.4,252,0.000,4690.5,100000,398452.4,0.5
6,2,/shipping,/shipping,347511.9,252,0.000,4887.4,0,347511.9,0.0
7,0,/checkout,/address,49100.7,8747,0.000,4978.5,200000,249100.7,3.0
8,1,/checkout,/checkout,1660054.5,11333,0.346,4966.4,0,1660054.5,0.0
9,2,/checkout,/creditcardinfo,24861.9,8747,0.000,5054.8,200000,224861.9,3.0


In [12]:
def plotTraceAgg(traceID):
    source = aggregateTraceDf[aggregateTraceDf['root'] == traceID]
    return alt.Chart(source, title='Aggregate Trace: ' + traceID).mark_bar().encode(
        y=alt.Y('Resource Name', type='nominal', sort=alt.SortField('Average order')),
        x = alt.X("Average loc start:Q", title= "Duration"),
        x2 = "Average loc end:Q",
        color = alt.Color('Error Rate:Q',sort='ascending'),
        tooltip = ['Average Duration:Q', 'Average Data Transfered:Q', 'Error Rate:Q']
    )

In [13]:
plotTraceAgg('/product')

alt.Chart(...)